# ***Parallelization using `SEND API`***

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct")

## Define the Report section Structure.

In [5]:
from pydantic import BaseModel, Field
from typing import TypedDict, List, Literal, Annotated

class Section(BaseModel):
    name: Annotated[str, Field(description="This is the name of the section of the report.")]
    description: Annotated[str, Field(description="The brief explanation of the main topics and concept of the section.")]
    
class Sections(BaseModel):
    sections: Annotated[List[Section], Field(description="Sections of the report.")]
    
planner = llm.with_structured_output(Sections)

In [7]:
planner.invoke("can you write a details report about the Agentic ai.").model_dump()

{'sections': [{'name': 'Introduction',
   'description': 'Agentic AI refers to a type of artificial intelligence that is capable of autonomous decision-making and action. It is designed to mimic human-like intelligence, with the ability to perceive, reason, and act in complex environments.'},
  {'name': 'Key Features',
   'description': 'Agentic AI systems are characterized by their ability to learn from experience, adapt to new situations, and make decisions based on their own goals and motivations. They are often designed to interact with humans and other agents in a collaborative or competitive manner.'},
  {'name': 'Applications',
   'description': 'Agentic AI has a wide range of applications, including robotics, autonomous vehicles, smart homes, and personalized assistants. It is also being explored for use in areas such as healthcare, finance, and education.'},
  {'name': 'Benefits',
   'description': 'The benefits of Agentic AI include improved efficiency, increased productivity